# Advanced preprocessing

# Table of Contents
- Load data
- Merge duplicated rows in `fcst` using mean / most recent fcst
- Convert the measure of `Cloud` in `fcst` to `obs`
- Interpolate fcst
- Convert dtype of 'time' in `energy`
- Merge fcst, obs, and energy
- Fill missing values in using ffill

## Load data
- from basic_processing.ipynb

In [39]:
import pickle
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [40]:
with open('pickles/dangjin_fcst_data.pkl','rb') as f:
    dangjin_fcst = pickle.load(f)
with open('pickles/dangjin_obs_data.pkl','rb') as f:
    dangjin_obs = pickle.load(f)
with open('pickles/ulsan_fcst_data.pkl','rb') as f:
    ulsan_fcst = pickle.load(f)
with open('pickles/ulsan_obs_data.pkl', 'rb') as f:
    ulsan_obs = pickle.load(f)

energy = pd.read_csv(glob('../original_dataset/energy.csv')[0])

## Merge duplicated rows in `fcst`
## Option 1(current Plan B): Merge duplicated rows in `fcst` using mean

In [41]:
print('total rows:',dangjin_fcst['time'].shape[0])
print('unique rows:', dangjin_fcst['time'].unique().shape[0])
print('total rows:',ulsan_fcst['time'].shape[0])
print('unique rows:',ulsan_fcst['time'].unique().shape[0])

total rows: 162208
unique rows: 8788
total rows: 162208
unique rows: 8788


In [42]:
dangjin_fcst_unique_mean = dangjin_fcst.groupby('time').mean()
ulsan_fcst_unique_mean = ulsan_fcst.groupby('time').mean()

dangjin_fcst_unique_mean = dangjin_fcst_unique_mean.reset_index() # because time was set to index
ulsan_fcst_unique_mean = ulsan_fcst_unique_mean.reset_index()

In [43]:
dangjin_fcst_unique_mean

,time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
0,2018-03-01 15:00:00,0.0,60.0,2.0,4.594039,-240.138102
1,2018-03-01 18:00:00,-2.0,60.0,1.0,4.932074,-225.872697
2,2018-03-01 21:00:00,-2.0,60.0,1.0,5.639833,-190.356464
3,2018-03-02 00:00:00,-2.0,52.5,1.0,5.709659,-136.663512
4,2018-03-02 03:00:00,-3.4,55.0,1.0,4.854618,-121.486735
...,...,...,...,...,...,...
8783,2021-03-03 12:00:00,7.0,40.0,1.0,-3.176148,-22.789567
8784,2021-03-03 15:00:00,8.0,40.0,1.0,-3.593860,-130.593860
8785,2021-03-03 18:00:00,5.0,55.0,1.0,-1.905256,-105.000000
8786,2021-03-03 21:00:00,1.0,80.0,1.0,-1.826397,45.204526


## Option 2(current Plan A): Merge duplicated rows in `fcst` using the most recent forecast

In [44]:
def merge_to_recent(data, time, due_hour = 21):
    '''
    @warning
    data에 'Forecast time'(기상 예측이 이루어진 시점) column이 존재해야 함.

    @description
    fcst에서 'time'이 중복되는 row들을 하나로 통합한다.
    이때, 가장 최근의 fcst data로 통합한다. (즉, 가장 최근의 fcst data 이외의 row는 모두 삭제한다.)
    '가장 최근'이라는 것은, 해당 'time'의 전날의 due_hour까지다. 
    예를 들어 2021-05-02 14:00:00(=time)를 예측하고, due_hour = 21이라면,
    2021-05-01 21:00:00 전까지의 row 중 가장 최근의 fcst로 통합한다.

    @parameters
    data: pd.DataFrame ; 통합이 이루어질 dataframe
    time: pd.Timestamp ; 통합하려는 time
    due_hour: int ; description 참고

    @return
    pd.Series ; 통합된 하나의 row

    '''
    # data only with the given time
    data_only_with_time = data.loc[data['time'] == time]
    
    # shift time
    time = time - pd.Timedelta(1, unit='days') # shifting 1 day to the past (yesterday)
    time = time.replace(hour=due_hour) # change 'hour' to DUE_HOUR

    # most recent forecast
    return data_only_with_time[data_only_with_time['Forecast time'] <= time].iloc[-1:,]


# example
TIME = pd.Timestamp('2018-03-03-21:00:00')
merge_to_recent(dangjin_fcst, TIME)

,time,Forecast time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
212,2018-03-03 21:00:00,2018-03-02 20:00:00,9.0,85.0,3.0,-1.704996,97.270042


In [45]:
# apply merge_to_recent (dangjin_fcst)

# create a new merged df
dangjin_fcst_unique_recent = pd.DataFrame()

for time in dangjin_fcst['time'].unique():
    to_append = merge_to_recent(dangjin_fcst, time)
    dangjin_fcst_unique_recent = dangjin_fcst_unique_recent.append(to_append)

# reset index and drop 'Forecast time'
dangjin_fcst_unique_recent =  dangjin_fcst_unique_recent.reset_index(drop=True)
dangjin_fcst_unique_recent =  dangjin_fcst_unique_recent.drop('Forecast time', axis=1)
dangjin_fcst_unique_recent

,time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
0,2018-03-02 00:00:00,-2.0,50.0,1.0,5.298564,-1.366635e+02
1,2018-03-02 03:00:00,-3.0,55.0,1.0,4.667902,-1.214867e+02
2,2018-03-02 06:00:00,-4.0,60.0,1.0,4.069439,-4.301988e+01
3,2018-03-02 09:00:00,-2.0,55.0,1.0,1.100000,-8.817457e-14
4,2018-03-02 12:00:00,2.0,45.0,1.0,-0.190809,-2.542414e+02
...,...,...,...,...,...,...
8780,2021-03-03 12:00:00,7.0,40.0,1.0,-3.176148,-2.278957e+01
8781,2021-03-03 15:00:00,8.0,40.0,1.0,-3.593860,-1.305939e+02
8782,2021-03-03 18:00:00,5.0,55.0,1.0,-1.905256,-1.050000e+02
8783,2021-03-03 21:00:00,1.0,80.0,1.0,-1.826397,4.520453e+01


In [46]:
# apply merge_to_recent (ulsan_fcst)

# create a new merged df
ulsan_fcst_unique_recent = pd.DataFrame()

for time in dangjin_fcst['time'].unique():
    to_append = merge_to_recent(ulsan_fcst, time)
    ulsan_fcst_unique_recent = ulsan_fcst_unique_recent.append(to_append)

# reset index and drop 'Forecast time'
ulsan_fcst_unique_recent =  ulsan_fcst_unique_recent.reset_index(drop=True)
ulsan_fcst_unique_recent =  ulsan_fcst_unique_recent.drop('Forecast time', axis=1)
ulsan_fcst_unique_recent

,time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
0,2018-03-02 00:00:00,0.0,40.0,2.0,1.010290,-202.011846
1,2018-03-02 03:00:00,-1.0,45.0,2.0,0.872657,-261.511292
2,2018-03-02 06:00:00,-1.0,45.0,1.0,1.006547,-252.027422
3,2018-03-02 09:00:00,0.0,35.0,1.0,1.225671,-205.692035
4,2018-03-02 12:00:00,6.0,25.0,1.0,1.160653,-182.296887
...,...,...,...,...,...,...
8780,2021-03-03 12:00:00,9.0,45.0,3.0,0.377795,82.381331
8781,2021-03-03 15:00:00,9.0,45.0,3.0,-1.075104,103.627427
8782,2021-03-03 18:00:00,8.0,55.0,3.0,-1.165822,103.461868
8783,2021-03-03 21:00:00,6.0,65.0,3.0,-0.590453,98.866955


## Convert the measure of `Cloud` in `fcst` to `obs`
- `fcst`와 `obs`에서 `Cloud`의 단위는 다르다.
- `fcst`와 `obs`를 따로 input으로 넣는 경우 크게 상관 없으나, 이 둘을 합쳐서 쓰는 경우에는 단위가 통일되는 것이 좋아 보인다.  
- fcst: 1-맑음, 2-구름보통, 3-구름많음, 4-흐림  
- obs: 10분위(0~10); 낮을 수록 구름 적음

### method
- Cloud_fcst -> Cloud_obs
- Cloud_fcst = x (x=1,2,3,4)일 때 Cloud_obs의 mean으로 변환

In [47]:
# get dictionary to convert from cloud_fcst to cloud_obs

# merge (note that this is not the 'real' merged data)
dangjin_merged_for_cloud = pd.merge(dangjin_fcst_unique_recent, dangjin_obs, on='time', suffixes=('_fcst','_obs'))
ulsan_merged_for_cloud = pd.merge(ulsan_fcst_unique_recent, ulsan_obs, on='time', suffixes=('_fcst','_obs'))

# mean
def cloud_from_fcst_to_obs(data):
    '''
    @parameter
    data: pd.DataFrame; data should contain both 'Cloud_fcst' and 'Cloud_obs'

    @return
    dictionary ; its key is the value of 'Cloud_fcst', its value is the value of 'Cloud_obs'
    '''

    # dropna
    data.dropna(inplace=True)
    # unique cloud_fcsts
    cloud_fcsts = data['Cloud_fcst'].unique()
    # dict to convert from fcst to obs
    convert = {cloud_fcst:0 for cloud_fcst in cloud_fcsts}

    # mean
    for cloud_fcst in cloud_fcsts:
        sliced_df = data[data['Cloud_fcst'] == cloud_fcst]
        convert[cloud_fcst] = sliced_df['Cloud_obs'].mean()

    return convert

dangjin_convert_cloud = cloud_from_fcst_to_obs(dangjin_merged_for_cloud)
ulsan_convert_cloud = cloud_from_fcst_to_obs(ulsan_merged_for_cloud)
dangjin_convert_cloud, ulsan_convert_cloud

({1.0: 2.7635372029606544,
  2.0: 3.8820678513731823,
  3.0: 6.18494516450648,
  4.0: 7.961345381526105},
 {2.0: 3.5910064239828694,
  1.0: 1.721059516023545,
  3.0: 6.145117540687161,
  4.0: 8.638197424892704})

In [48]:
# apply the convert-dictionary
dangjin_fcst_unique_recent['Cloud'].replace(dangjin_convert_cloud, inplace=True)
ulsan_fcst_unique_recent['Cloud'].replace(ulsan_convert_cloud, inplace=True)
display(dangjin_fcst_unique_recent['Cloud'])
display(ulsan_fcst_unique_recent['Cloud'])

0       2.763537
1       2.763537
2       2.763537
3       2.763537
4       2.763537
          ...   
8780    2.763537
8781    2.763537
8782    2.763537
8783    2.763537
8784    2.763537
Name: Cloud, Length: 8785, dtype: float64

0       3.591006
1       3.591006
2       1.721060
3       1.721060
4       1.721060
          ...   
8780    6.145118
8781    6.145118
8782    6.145118
8783    6.145118
8784    6.145118
Name: Cloud, Length: 8785, dtype: float64

## Interpolate fcst
- code from @김진수

In [49]:
# code from @김진수
# linear interpolation

def interpolation(df):

    df_copy = df.copy()
    var_names = df.columns

    total_s = list()
    time_list = list()
    
    for var_name in var_names:
        s = list()
        for i in range(df_copy.shape[0] - 1):
            timedeltas = df_copy["time"][i+1] - df_copy["time"][i]
            n_intervals = int(timedeltas / np.timedelta64(1, "h"))

            for j in range(n_intervals):
        
                if var_name == "time":
                    time_stamps = df_copy["time"][i] + timedeltas * j / n_intervals
                    time_list.append(time_stamps)
                else:
                    add_ = df_copy[var_name][i] + (df_copy[var_name][i+1] - df_copy[var_name][i]) / n_intervals * j
                    s.append(add_)

        if var_name == "time":
            time_list = np.array(time_list).reshape(-1,1)
            total_s.append(time_list)
        else:
            s = np.array(s).reshape(-1,1)
            total_s.append(s)

    total_s = np.array(total_s).T.reshape(-1, len(var_names))
    df_converted = pd.DataFrame(total_s, columns = var_names)

    return df_converted

In [50]:
dangjin_fcst_interpolated = interpolation(dangjin_fcst_unique_recent)
ulsan_fcst_interpolated = interpolation(ulsan_fcst_unique_recent)
dangjin_obs_interpolated = interpolation(dangjin_obs)
ulsan_obs_interpolated = interpolation(ulsan_obs)

## Convert dtype of 'time' in `energy`
- 'time' of `energy` is 'object'. 
- However, `pd.to_datetime` doesn't accepts '24:00:00' as an hour. Instead accepts '00:00:00'.
- Therefore, custom function is needed.

In [55]:
def to_datetime_for_energy(date_str):
    if date_str[11:13] == '24':
        date_str = date_str[0:11] + '00' + date_str[13:]
        return pd.to_datetime(date_str) + pd.Timedelta(1, unit='days')

    else:
        return pd.to_datetime(date_str)

print(to_datetime_for_energy('2018-03-01 24:00:00'))

2018-03-02 00:00:00


In [57]:
energy['time'] = energy['time'].apply(to_datetime_for_energy)

## Merge fcst, obs, and energy
- inner merge on 'time'

In [58]:
dangjin_merged_ = pd.merge(dangjin_fcst_interpolated, dangjin_obs_interpolated, on='time', suffixes=('_obs','_fcst'))
dangjin_merged = pd.merge(dangjin_merged_, energy.loc[:,['time','dangjin_floating','dangjin_warehouse','dangjin']], on='time')

ulsan_merged_ = pd.merge(ulsan_fcst_interpolated, ulsan_obs_interpolated, on='time', suffixes=('_obs','_fcst'))
ulsan_merged = pd.merge(ulsan_merged_, energy.loc[:,['time','ulsan']], on='time')

display(dangjin_merged)
display(ulsan_merged)

,time,Temperature_obs,Humidity_obs,Cloud_obs,Wind_X_obs,Wind_Y_obs,Temperature_fcst,Humidity_fcst,Cloud_fcst,Wind_X_fcst,Wind_Y_fcst,dangjin_floating,dangjin_warehouse,dangjin
0,2018-03-02 00:00:00,-2.0,50.0,2.763537,5.298564,-136.663512,-2.7,56.0,NaN,3.7,-0.0,0.0,0.0,0
1,2018-03-02 01:00:00,-2.333333,51.666667,2.763537,5.088343,-131.604586,-2.8,53.0,NaN,3.8,-0.0,0.0,0.0,0
2,2018-03-02 02:00:00,-2.666667,53.333333,2.763537,4.878123,-126.545661,-3.2,52.0,NaN,2.5,-0.0,0.0,0.0,0
3,2018-03-02 03:00:00,-3.0,55.0,2.763537,4.667902,-121.486735,-3.5,50.0,NaN,2.443201,6.840403,0.0,0.0,0
4,2018-03-02 04:00:00,-3.333333,56.666667,2.763537,4.468414,-95.331116,-4.2,55.0,NaN,0.899903,38.302222,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25602,2021-01-31 18:00:00,6.0,75.0,7.961345,-3.374657,-22.789567,7.8,70.0,8.0,-1.879385,-68.404029,10.0,6.0,0
25603,2021-01-31 19:00:00,6.0,76.666667,7.961345,-3.521363,-3.54999,6.7,75.0,8.0,-1.409539,-68.404029,0.0,0.0,0
25604,2021-01-31 20:00:00,6.0,78.333333,7.961345,-3.668069,15.689587,6.2,77.0,8.0,-0.751754,-68.404029,0.0,0.0,0
25605,2021-01-31 21:00:00,6.0,80.0,7.961345,-3.814776,34.929164,5.3,82.0,8.0,-0.449951,-176.190222,0.0,0.0,0


,time,Temperature_obs,Humidity_obs,Cloud_obs,Wind_X_obs,Wind_Y_obs,Temperature_fcst,Humidity_fcst,Cloud_fcst,Wind_X_fcst,Wind_Y_fcst,ulsan
0,2018-03-02 00:00:00,0.0,40.0,3.591006,1.01029,-202.011846,1.3,37.0,NaN,3.194955,-116.286849,0
1,2018-03-02 01:00:00,-0.333333,41.666667,3.591006,0.964412,-221.844994,0.5,42.0,NaN,2.0,-0.0,0
2,2018-03-02 02:00:00,-0.666667,43.333333,3.591006,0.918535,-241.678143,-0.4,46.0,NaN,2.3,-0.0,0
3,2018-03-02 03:00:00,-1.0,45.0,3.591006,0.872657,-261.511292,-0.9,48.0,9.0,1.879385,6.840403,0
4,2018-03-02 04:00:00,-1.0,45.0,2.967691,0.917287,-258.350002,-1.5,51.0,8.0,2.161293,-116.286849,0
...,...,...,...,...,...,...,...,...,...,...,...,...
25602,2021-01-31 18:00:00,9.0,55.0,8.638197,-2.472747,-61.185365,9.3,43.0,0.0,-3.4,0.0,8
25603,2021-01-31 19:00:00,8.666667,60.0,8.638197,-2.629993,-75.790243,8.8,50.0,5.0,-2.349232,-68.404029,0
25604,2021-01-31 20:00:00,8.333333,65.0,8.638197,-2.78724,-90.395122,8.7,49.0,1.0,-3.664801,-68.404029,0
25605,2021-01-31 21:00:00,8.0,70.0,8.638197,-2.944486,-105.0,8.4,51.0,7.0,-1.54269,-176.190222,0


## Fill missing values in using ffill
- fillna with bfill(backward fill)
- if bfill is not available, use ffill(forward fill) instead

In [60]:
# keep the original
dangjin_merged_with_na = dangjin_merged.copy()
ulsan_merged_with_na = ulsan_merged.copy()

In [61]:
# check missing values
print(dangjin_merged_with_na.isna().sum())
print(ulsan_merged_with_na.isna().sum())

time                    0
Temperature_obs         0
Humidity_obs            0
Cloud_obs               0
Wind_X_obs              0
Wind_Y_obs              0
Temperature_fcst       47
Humidity_fcst          43
Cloud_fcst           4077
Wind_X_fcst            44
Wind_Y_fcst            44
dangjin_floating       24
dangjin_warehouse      48
dangjin                 0
dtype: int64
time                   0
Temperature_obs        0
Humidity_obs           0
Cloud_obs              0
Wind_X_obs             0
Wind_Y_obs             0
Temperature_fcst       7
Humidity_fcst          2
Cloud_fcst          1251
Wind_X_fcst            2
Wind_Y_fcst            2
ulsan                  0
dtype: int64


In [62]:
# fill
dangjin_merged = dangjin_merged.fillna(method='bfill')
ulsan_merged = ulsan_merged.fillna(method='bfill')
display(dangjin_merged)
display(ulsan_merged)

,time,Temperature_obs,Humidity_obs,Cloud_obs,Wind_X_obs,Wind_Y_obs,Temperature_fcst,Humidity_fcst,Cloud_fcst,Wind_X_fcst,Wind_Y_fcst,dangjin_floating,dangjin_warehouse,dangjin
0,2018-03-02 00:00:00,-2.000000,50.000000,2.763537,5.298564,-136.663512,-2.7,56.0,4.0,3.700000,-8.817457e-14,0.0,0.0,0
1,2018-03-02 01:00:00,-2.333333,51.666667,2.763537,5.088343,-131.604586,-2.8,53.0,4.0,3.800000,-8.817457e-14,0.0,0.0,0
2,2018-03-02 02:00:00,-2.666667,53.333333,2.763537,4.878123,-126.545661,-3.2,52.0,4.0,2.500000,-8.817457e-14,0.0,0.0,0
3,2018-03-02 03:00:00,-3.000000,55.000000,2.763537,4.667902,-121.486735,-3.5,50.0,4.0,2.443201,6.840403e+00,0.0,0.0,0
4,2018-03-02 04:00:00,-3.333333,56.666667,2.763537,4.468414,-95.331116,-4.2,55.0,4.0,0.899903,3.830222e+01,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25602,2021-01-31 18:00:00,6.000000,75.000000,7.961345,-3.374657,-22.789567,7.8,70.0,8.0,-1.879385,-6.840403e+01,10.0,6.0,0
25603,2021-01-31 19:00:00,6.000000,76.666667,7.961345,-3.521363,-3.549990,6.7,75.0,8.0,-1.409539,-6.840403e+01,0.0,0.0,0
25604,2021-01-31 20:00:00,6.000000,78.333333,7.961345,-3.668069,15.689587,6.2,77.0,8.0,-0.751754,-6.840403e+01,0.0,0.0,0
25605,2021-01-31 21:00:00,6.000000,80.000000,7.961345,-3.814776,34.929164,5.3,82.0,8.0,-0.449951,-1.761902e+02,0.0,0.0,0


,time,Temperature_obs,Humidity_obs,Cloud_obs,Wind_X_obs,Wind_Y_obs,Temperature_fcst,Humidity_fcst,Cloud_fcst,Wind_X_fcst,Wind_Y_fcst,ulsan
0,2018-03-02 00:00:00,0.000000,40.000000,3.591006,1.010290,-202.011846,1.3,37.0,9.0,3.194955,-1.162868e+02,0
1,2018-03-02 01:00:00,-0.333333,41.666667,3.591006,0.964412,-221.844994,0.5,42.0,9.0,2.000000,-8.817457e-14,0
2,2018-03-02 02:00:00,-0.666667,43.333333,3.591006,0.918535,-241.678143,-0.4,46.0,9.0,2.300000,-8.817457e-14,0
3,2018-03-02 03:00:00,-1.000000,45.000000,3.591006,0.872657,-261.511292,-0.9,48.0,9.0,1.879385,6.840403e+00,0
4,2018-03-02 04:00:00,-1.000000,45.000000,2.967691,0.917287,-258.350002,-1.5,51.0,8.0,2.161293,-1.162868e+02,0
...,...,...,...,...,...,...,...,...,...,...,...,...
25602,2021-01-31 18:00:00,9.000000,55.000000,8.638197,-2.472747,-61.185365,9.3,43.0,0.0,-3.400000,2.204364e-14,8
25603,2021-01-31 19:00:00,8.666667,60.000000,8.638197,-2.629993,-75.790243,8.8,50.0,5.0,-2.349232,-6.840403e+01,0
25604,2021-01-31 20:00:00,8.333333,65.000000,8.638197,-2.787240,-90.395122,8.7,49.0,1.0,-3.664801,-6.840403e+01,0
25605,2021-01-31 21:00:00,8.000000,70.000000,8.638197,-2.944486,-105.000000,8.4,51.0,7.0,-1.542690,-1.761902e+02,0


## Pickle dfs

In [63]:
with open('pickles/dangjin_merged.pkl','wb') as f:
    pickle.dump(dangjin_merged,f)
with open('pickles/ulsan_merged.pkl','wb') as f:
    pickle.dump(ulsan_merged,f)